<a href="https://colab.research.google.com/github/minomugeshi/watermark/blob/main/GoogleColab%E5%AE%9F%E8%A1%8C%E5%8F%AF%E5%90%A6%E7%A2%BA%E8%AA%8D%E7%94%A8_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
a = 100 + 29
print(a)

129


In [ ]:
from google.colab import drive

# Google Driveをマウント
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title　PDFファイルにウォーターマークを追記する。
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
import io

def add_watermark(pdf_path, watermark_text, font_size=40, color=(0.5, 0.5, 0.5), opacity=0.5):
    """Adds a watermark to the specified PDF."""
    try:
        # Read the existing PDF
        with open(pdf_path, "rb") as f:
            reader = PdfReader(f)
            writer = PdfWriter()

            # Create a watermark canvas
            packet = io.BytesIO()
            can = canvas.Canvas(packet)
            can.setFont("Helvetica", font_size)
            can.setFillColorRGB(color[0], color[1], color[2], alpha=opacity)
            can.save()

            # Move to the beginning of the StringIO buffer
            packet.seek(0)
            watermark = PdfReader(packet)

            # Add watermark to each page
            for page_number in range(len(reader.pages)):
                page = reader.pages[page_number]
                packet.seek(0)
                page_width = float(page.mediabox.width)
                page_height = float(page.mediabox.height)
                can = canvas.Canvas(packet, pagesize=(page_width, page_height))
                can.setFont("Helvetica", font_size)
                can.setFillColorRGB(color[0], color[1], color[2], alpha=opacity)
                can.drawString(page_width - can.stringWidth(watermark_text, "Helvetica", font_size) - 10,
                               page_height - font_size - 10,
                               watermark_text)
                can.save()

                # Move to the beginning of the StringIO buffer
                packet.seek(0)
                watermark = PdfReader(packet)
                page.merge_page(watermark.pages[0])
                writer.add_page(page)

            # Save the watermarked PDF
            with open(f"{pdf_path}_watermarked.pdf", "wb") as output_stream:
                writer.write(output_stream)

    except FileNotFoundError:
        print(f"Error: PDF file not found: {pdf_path}")

# Example usage (replace with your actual file path and desired settings)
pdf_path = "/content/drive/MyDrive/出図IN/your_actual_pdf_file.pdf"
watermark_text = "Sample Watermark"
font_size = 50  # Font size
color = (0.0, 0.0, 1.0)  # RGB color (blue)
opacity = 0.3  # Opacity (0.0 to 1.0)

add_watermark(pdf_path, watermark_text, font_size, color, opacity)


In [8]:
# @title ウォーターマーク印字テスト

# インストールが必要なライブラリ
print("初期化")
!pip install -q PyPDF2
!pip install -q reportlab
!pip install -q gspread
!pip install -q oauth2client
!pip install -q google-api-python-client

from google.colab import drive
import os
import PyPDF2
from reportlab.pdfgen import canvas
import io
from IPython.display import display
from ipywidgets import widgets, VBox, Output
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define directories
input_dir = '/content/drive/MyDrive/出図IN'
output_dir = '/content/drive/MyDrive/出図OUT'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Function to list PDF files
def list_pdf_files():
    return [f for f in os.listdir(input_dir) if f.endswith('.pdf')]

# Function to add watermark
def add_watermark(pdf_path, watermark_text):
    try:
        with open(pdf_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            writer = PyPDF2.PdfWriter()

            packet = io.BytesIO()
            can = canvas.Canvas(packet)
            can.setFont("Helvetica", 10)
            can.setFillColorRGB(0.5, 0.5, 0.5, alpha=0.5)
            can.save()

            packet.seek(0)
            watermark = PyPDF2.PdfReader(packet)

            for page_number in range(len(reader.pages)):
                page = reader.pages[page_number]
                packet.seek(0)
                page_width = float(page.mediabox.width)
                page_height = float(page.mediabox.height)
                can = canvas.Canvas(packet, pagesize=(page_width, page_height))
                can.setFont("Helvetica", 10)
                can.setFillColorRGB(0.5, 0.5, 0.5, alpha=0.5)
                can.drawString(page_width - can.stringWidth(watermark_text, "Helvetica", 10) - 10,
                               page_height - 10,
                               watermark_text)
                can.save()

                packet.seek(0)
                watermark = PyPDF2.PdfReader(packet)
                page.merge_page(watermark.pages[0])
                writer.add_page(page)

            output_filename = f"({watermark_text}){os.path.basename(pdf_path)}"
            output_path = os.path.join(output_dir, output_filename)
            with open(output_path, "wb") as output_stream:
                writer.write(output_stream)
            return output_filename

    except FileNotFoundError:
        print(f"Error: PDF file not found: {pdf_path}")
        return None

# Function to create Google Sheets
def create_spreadsheet(watermarked_files):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/true-solstice-247904-c27bb98cc923.json', scope)
    client = gspread.authorize(creds)

    spreadsheet = client.create('ウォーターマーク')
    worksheet = spreadsheet.get_worksheet(0)
    worksheet.update_title('Watermarked Files')

    header = ['番号', 'ファイル名']
    data = [[str(i + 1), filename] for i, filename in enumerate(watermarked_files)]
    worksheet.append_row(header)
    worksheet.append_rows(data)

    # Move spreadsheet to the specified folder
    drive_service = build('drive', 'v3', credentials=creds)
    file_id = spreadsheet.id
    folder_id = '18VZxec1sEVt_BAFDJ5-aZcosT5rDXBdc'  # Replace with your output folder ID
    try:
        file = drive_service.files().get(fileId=file_id, fields='parents').execute()
        previous_parents = ",".join(file.get('parents'))
        drive_service.files().update(fileId=file_id, addParents=folder_id, removeParents=previous_parents).execute()
    except HttpError as error:
        print(f"An error occurred: {error}")
        print(f"File ID: {file_id}")
        print(f"Folder ID: {folder_id}")

# UI elements
output = Output()

def on_load_button_clicked(b):
    with output:
        output.clear_output()
        pdf_files = list_pdf_files()
        if pdf_files:
            print(f"Found {len(pdf_files)} PDF files:")
            for pdf in pdf_files:
                print(pdf)
            show_all_select_button()
        else:
            print("No PDF files found in '出図IN' folder.")

def show_all_select_button():
    all_select_button = widgets.Button(description="全選択")

    def on_all_select_button_clicked(b):
        with output:
            output.clear_output()
            selected_pdfs = list_pdf_files()
            print(f"Selected {len(selected_pdfs)} PDF files.")
            show_watermark_input(selected_pdfs)

    all_select_button.on_click(on_all_select_button_clicked)
    display(all_select_button)

def show_watermark_input(selected_pdfs):
    watermark_text_widget = widgets.Text(
        description='Watermark:',
        disabled=False
    )
    confirm_button = widgets.Button(description="印字")

    def on_confirm_button_clicked(b):
        global watermark_text
        watermark_text = watermark_text_widget.value

        # Check if watermark_text contains Japanese or full-width characters
        if re.search(r'[^\x00-\x7F]', watermark_text):
            with output:
                output.clear_output()
                print("Error: ウォーターマークには日本語や全角文字を使用しないでください。")
                show_watermark_input(selected_pdfs)
        elif len(watermark_text) == 0:
            with output:
                output.clear_output()
                print("Error: ウォーターマークを入力してください。")
                show_watermark_input(selected_pdfs)
        else:
            with output:
                output.clear_output()
                print(f"入力されたウォーターマーク: {watermark_text}")
                yes_button = widgets.Button(description="YES")
                no_button = widgets.Button(description="NO")

                def on_yes_button_clicked(yes_b):
                    with output:
                        output.clear_output()
                        print("印字処理")
                        watermarked_files = []
                        for pdf in selected_pdfs:
                            pdf_path = os.path.join(input_dir, pdf)
                            output_filename = add_watermark(pdf_path, watermark_text)
                            if output_filename:
                                watermarked_files.append(output_filename)
                        print(f"Watermarked {len(watermarked_files)} files.")
                        print("Files:")
                        for file in watermarked_files:
                            print(file)

                        # Create the spreadsheet
                        create_spreadsheet(watermarked_files)

                        print("印字完了")

                def on_no_button_clicked(no_b):
                    with output:
                        output.clear_output()
                        show_watermark_input(selected_pdfs)

                yes_button.on_click(on_yes_button_clicked)
                no_button.on_click(on_no_button_clicked)
                display(VBox([widgets.Label(value='これでOKですか？'), yes_button, no_button]))

    confirm_button.on_click(on_confirm_button_clicked)
    display(VBox([watermark_text_widget, confirm_button]))

# Display load button
load_button = widgets.Button(description="読み込み")
load_button.on_click(on_load_button_clicked)
display(load_button)
display(output)


初期化
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Button(description='読み込み', style=ButtonStyle())

Output()

In [3]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

from googleapiclient.discovery import build
from google.oauth2 import service_account

# サービスアカウントのJSONファイルへのパス
SERVICE_ACCOUNT_FILE = '/content/drive/MyDrive/true-solstice-247904-c27bb98cc923.json'
# フォルダーID
FOLDER_ID = '18VZxec1sEVt_BAFDJ5-aZcosT5rDXBdc'

# サービスアカウントのクレデンシャルを読み込む
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])

# Google Drive APIサービスを構築
service = build('drive', 'v3', credentials=credentials)

# フォルダーの共有設定を更新して、サービスアカウントに編集権限を付与
def share_folder_with_service_account(service, folder_id, email):
    permissions = {
        'type': 'user',
        'role': 'writer',
        'emailAddress': email
    }
    try:
        service.permissions().create(
            fileId=folder_id,
            body=permissions,
            fields='id'
        ).execute()
        print(f"フォルダー {folder_id} が {email} と共有されました。")
    except Exception as e:
        print(f"フォルダーの共有に失敗しました: {e}")

# サービスアカウントのメールアドレスを取得
service_account_email = credentials.service_account_email

# フォルダーをサービスアカウントと共有
share_folder_with_service_account(service, FOLDER_ID, service_account_email)


フォルダーの共有に失敗しました: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/18VZxec1sEVt_BAFDJ5-aZcosT5rDXBdc/permissions?fields=id&alt=json returned "File not found: 18VZxec1sEVt_BAFDJ5-aZcosT5rDXBdc.". Details: "[{'message': 'File not found: 18VZxec1sEVt_BAFDJ5-aZcosT5rDXBdc.', 'domain': 'global', 'reason': 'notFound', 'location': 'fileId', 'locationType': 'parameter'}]">
